# Bronze - Latest Measurements Ingestion

**Purpose**: Fetch latest air quality measurements from OpenAQ API for active Canadian monitoring stations\
**Schedule**: Every hour\
**Source**: OpenAQ API v3 `/locations/{id}/latest` endpoint\
**Target**: `airquality.bronze.measurements`

### 1.0 - Configuration

In [0]:
%run ./00_utils

### 2.0 - API Call Functions

In [0]:
import time

def fetch_all_latest_measurements(location_ids):
    """
    Fetch latest measurements for all locations.
    
    Args:
        location_ids: List of OpenAQ location IDs
        
    Returns:
        List of measurement dictionaries
        
    Note:
        Includes 1 second delay between calls to respect API rate limit (60/min)
    """
    latest_measurements = []
    
    for i, loc_id in enumerate(location_ids):
        try:
            data = fetch_openaq(f"locations/{loc_id}/latest")
            
            for result in data.get("results", []):
                latest_measurements.append(result)
            
            if (i + 1) % 50 == 0:
                print(f"Processed {i + 1}/{len(location_ids)} locations")
            
            time.sleep(1)
            
        except Exception as e:
            print(f"Error fetching location {loc_id}: {e}")
            continue
    
    return latest_measurements

### 3.0 - Run ingestion

In [0]:
from datetime import datetime, timedelta, timezone

# Get only locations updated in the last 24 hours, taking the latest version of each
cutoff = (datetime.now(timezone.utc) - timedelta(hours=24)).strftime("%Y-%m-%dT%H:%M:%SZ")
print(f"Cutoff: {cutoff}")

# Define the SQL query to fetch the latest versions of each location
ids_query = f"""
WITH filtered AS (
    SELECT
        id,
        ingested_at,
        ROW_NUMBER() OVER (
            PARTITION BY id
            ORDER BY ingested_at DESC
        ) AS rn
    FROM {CATALOG}.{SCHEMA}.locations
    WHERE get_json_object(datetimeLast, '$.utc') >= '{cutoff}'
)
SELECT id
FROM filtered
WHERE rn = 1
"""

# Execute the query and get a Spark DataFrame with the location IDs
df_ids = spark.sql(ids_query)

# Extract the location IDs
location_ids = [row.id for row in df_ids.collect()]

print(f"Fetching latest for {len(location_ids)} active locations...")
print(f"Estimated time: ~{len(location_ids) // 60 + 1} minutes")

# Fetch latest measurements for each location
measurements = fetch_all_latest_measurements(location_ids)

print(f"\nTotal measurements fetched: {len(measurements)}")

### 4.0 - Clean Data

In [0]:
import json

# Clean measurements - convert nested fields to JSON strings
measurements_clean = []
for m in measurements:
    clean = {
        "sensors_id": m.get("sensorsId"),
        "locations_id": m.get("locationsId"),
        "value": m.get("value"),
        "datetime": json.dumps(m.get("datetime")),
        "coordinates": json.dumps(m.get("coordinates")),
    }
    measurements_clean.append(clean)

print(f"Cleaned {len(measurements_clean)} measurements")

### 5.0 - Save Measuraments to Bronze 

In [0]:
import pandas as pd
from pyspark.sql import functions as F

# Create DataFrame and add metadata
pdf = pd.DataFrame(measurements_clean)
df = spark.createDataFrame(pdf)

df_bronze = df.withColumn("ingested_at", F.lit(datetime.now(timezone.utc).isoformat())) \
              .withColumn("source", F.lit("openaq_api_v3_latest"))

# Append to bronze table
df_bronze.write.mode("append").saveAsTable(f"{CATALOG}.{SCHEMA}.measurements")

print(f"Saved {df_bronze.count()} measurements to {CATALOG}.{SCHEMA}.measurements")

### 6.0 - Check Measuraments Table

In [0]:
spark.sql("SHOW TABLES IN airquality.bronze").display()

In [0]:
spark.sql("DESCRIBE TABLE EXTENDED airquality.bronze.measurements").display()